In [213]:
import os
import pandas as pd
import numpy as np
import warnings; warnings.simplefilter('ignore')

In [214]:
## load data per year
Year_1978 = pd.read_csv('./gl1990-2018/GL1978.TXT',sep=',',header=None)
Year_1979 = pd.read_csv('./gl1990-2018/GL1979.TXT',sep=',',header=None)
Year_1980 = pd.read_csv('./gl1990-2018/GL1980.TXT',sep=',',header=None)
Year_1981 = pd.read_csv('./gl1990-2018/GL1981.TXT',sep=',',header=None)
Year_1982 = pd.read_csv('./gl1990-2018/GL1982.TXT',sep=',',header=None)
Year_1983 = pd.read_csv('./gl1990-2018/GL1983.TXT',sep=',',header=None)
Year_1984 = pd.read_csv('./gl1990-2018/GL1984.TXT',sep=',',header=None)
Year_1985 = pd.read_csv('./gl1990-2018/GL1985.TXT',sep=',',header=None)
Year_1986 = pd.read_csv('./gl1990-2018/GL1986.TXT',sep=',',header=None)
Year_1987 = pd.read_csv('./gl1990-2018/GL1987.TXT',sep=',',header=None)
Year_1988 = pd.read_csv('./gl1990-2018/GL1988.TXT',sep=',',header=None)
Year_1989 = pd.read_csv('./gl1990-2018/GL1989.TXT',sep=',',header=None)
Year_1990 = pd.read_csv('./gl1990-2018/GL1990.TXT',sep=',',header=None)
Year_1991 = pd.read_csv('./gl1990-2018/GL1991.TXT',sep=',',header=None)
Year_1992 = pd.read_csv('./gl1990-2018/GL1992.TXT',sep=',',header=None)
Year_1993 = pd.read_csv('./gl1990-2018/GL1993.TXT',sep=',',header=None)
Year_1994 = pd.read_csv('./gl1990-2018/GL1994.TXT',sep=',',header=None)
Year_1995 = pd.read_csv('./gl1990-2018/GL1995.TXT',sep=',',header=None)
Year_1996 = pd.read_csv('./gl1990-2018/GL1996.TXT',sep=',',header=None)
Year_1997 = pd.read_csv('./gl1990-2018/GL1997.TXT',sep=',',header=None)
Year_1998 = pd.read_csv('./gl1990-2018/GL1998.TXT',sep=',',header=None)
Year_1999 = pd.read_csv('./gl1990-2018/GL1999.TXT',sep=',',header=None)
Year_2000 = pd.read_csv('./gl1990-2018/GL2000.TXT',sep=',',header=None)
Year_2001 = pd.read_csv('./gl1990-2018/GL2001.TXT',sep=',',header=None)
Year_2002 = pd.read_csv('./gl1990-2018/GL2002.TXT',sep=',',header=None)
Year_2003 = pd.read_csv('./gl1990-2018/GL2003.TXT',sep=',',header=None)
Year_2004 = pd.read_csv('./gl1990-2018/GL2004.TXT',sep=',',header=None)
Year_2005 = pd.read_csv('./gl1990-2018/GL2005.TXT',sep=',',header=None)
Year_2006 = pd.read_csv('./gl1990-2018/GL2006.TXT',sep=',',header=None)
Year_2007 = pd.read_csv('./gl1990-2018/GL2007.TXT',sep=',',header=None)
Year_2008 = pd.read_csv('./gl1990-2018/GL2008.TXT',sep=',',header=None)
Year_2009 = pd.read_csv('./gl1990-2018/GL2009.TXT',sep=',',header=None)
Year_2010 = pd.read_csv('./gl1990-2018/GL2010.TXT',sep=',',header=None)
Year_2011 = pd.read_csv('./gl1990-2018/GL2011.TXT',sep=',',header=None)
Year_2012 = pd.read_csv('./gl1990-2018/GL2012.TXT',sep=',',header=None)
Year_2013 = pd.read_csv('./gl1990-2018/GL2013.TXT',sep=',',header=None)
Year_2014 = pd.read_csv('./gl1990-2018/GL2014.TXT',sep=',',header=None)
Year_2015 = pd.read_csv('./gl1990-2018/GL2015.TXT',sep=',',header=None)
Year_2016 = pd.read_csv('./gl1990-2018/GL2016.TXT',sep=',',header=None)
Year_2017 = pd.read_csv('./gl1990-2018/GL2017.TXT',sep=',',header=None)
Year_2018 = pd.read_csv('./gl1990-2018/GL2018.TXT',sep=',',header=None)

In [215]:
alldata = pd.concat([Year_1978, Year_1979, Year_1980, Year_1981, Year_1982, Year_1983, Year_1984, Year_1985, Year_1986, Year_1987, Year_1988, Year_1989, Year_1990, Year_1991, Year_1992, Year_1993, Year_1994, Year_1995, Year_1996, Year_1997, Year_1998, Year_1999, Year_2000, Year_2001, Year_2002, Year_2003, Year_2004, Year_2005, Year_2006, Year_2007, Year_2008, Year_2009, Year_2010, Year_2011, Year_2012, Year_2013, Year_2014, Year_2015, Year_2016, Year_2017, Year_2018])

In [216]:
alldata.head()

,0,1,2,3,4,5,6,7,8,9,...,151,152,153,154,155,156,157,158,159,160
0,19780405,0,Wed,MIN,AL,1,SEA,AL,1,2,...,John Hale,9.0,stinb101,Bob Stinson,2.0,reync001,Craig Reynolds,6.0,NaN,Y
1,19780406,0,Thu,MIN,AL,2,SEA,AL,2,5,...,John Hale,9.0,stinb101,Bob Stinson,2.0,reync001,Craig Reynolds,6.0,NaN,Y
2,19780406,0,Thu,HOU,NL,1,CIN,NL,1,9,...,Dave Concepcion,6.0,geroc101,Cesar Geronimo,8.0,seavt001,Tom Seaver,1.0,NaN,Y
3,19780407,0,Fri,OAK,AL,1,CAL,AL,1,0,...,Dave Chalk,5.0,mullr001,Rance Mulliniks,6.0,humpt101,Terry Humphrey,2.0,NaN,Y
4,19780407,0,Fri,BOS,AL,1,CHA,AL,1,5,...,Eric Soderholm,5.0,nordw101,Wayne Nordhagen,2.0,kessd101,Don Kessinger,6.0,NaN,Y


In [217]:
##BAL, BOS, NYA, TOR

#Yankees_data = alldata[(alldata[6] == 'BAL') | (alldata[alldata[3] == 'BAL')]
Yankees_data = alldata.loc[(alldata[6] == 'TOR') | (alldata[3] == 'TOR')]                                             

In [218]:
Yankees_data[0]=pd.to_datetime(Yankees_data[0],format='%Y%m%d')  ## convert to datetime
Yankees_data.set_index(0, inplace = True) ## set datetime as index

In [219]:
Yankees_data = Yankees_data[[1, 2, 3, 4, 6, 12, 13, 17, 23,51, 9, 10]]
Yankees_data.columns = ['GameType','DayOfWeek','VisitingTeam_Team','VisitingTeam_League','HomeTeam','DayNight','Completion','Attendance','VisitingTeam_Hits','HomeTeam_Hits','VisitingScore','HomeScore']


In [220]:
Yankees_data.head()

,GameType,DayOfWeek,VisitingTeam_Team,VisitingTeam_League,HomeTeam,DayNight,Completion,Attendance,VisitingTeam_Hits,HomeTeam_Hits,VisitingScore,HomeScore
0,,,,,,,,,,,,
1978-04-07,0,Fri,TOR,AL,DET,D,NaN,52528.0,0.0,1.0,2,6
1978-04-08,0,Sat,TOR,AL,DET,D,NaN,12413.0,1.0,2.0,5,2
1978-04-09,0,Sun,TOR,AL,DET,D,NaN,9169.0,2.0,1.0,4,8
1978-04-11,0,Tue,TOR,AL,CHA,D,NaN,3347.0,1.0,1.0,5,9
1978-04-12,0,Wed,TOR,AL,CHA,D,NaN,5742.0,2.0,1.0,4,5


In [221]:
Yankees_data['GameType'].value_counts()

0    6315
2     103
1     103
Name: GameType, dtype: int64

In [222]:
## Create Winner column based PointDifferential
Yankees_data = Yankees_data.loc[Yankees_data['GameType'] != 1] ## Exclude one game of double-header that has no attendance

Yankees_data.loc[Yankees_data['HomeTeam'] =='TOR','PointDifferential'] = Yankees_data['HomeScore'] - Yankees_data['VisitingScore']
Yankees_data.loc[(Yankees_data['HomeTeam'] =='TOR') & (Yankees_data['PointDifferential'] > 0), 'Winner'] = 'W'
Yankees_data.loc[(Yankees_data['HomeTeam'] =='TOR') & (Yankees_data['PointDifferential'] < 0), 'Winner'] = 'L'

Yankees_data.loc[Yankees_data['VisitingTeam_Team'] =='TOR','PointDifferential'] = Yankees_data['VisitingScore'] - Yankees_data['HomeScore']
Yankees_data.loc[(Yankees_data['VisitingTeam_Team'] =='TOR') & (Yankees_data['PointDifferential'] > 0), 'Winner'] = 'W'
Yankees_data.loc[(Yankees_data['VisitingTeam_Team'] =='TOR') & (Yankees_data['PointDifferential'] < 0), 'Winner'] = 'L'


#Yankees_data['PointDifferential'] = Yankees_data['HomeScore'] - Yankees_data['VisitingScore']
#Yankees_data.loc[Yankees_data['PointDifferential'] > 0, 'Winner'] = 'W'
#Yankees_data.loc[Yankees_data['PointDifferential'] < 0, 'Winner'] = 'L'


In [223]:
## Assign a game number for each game of each season
gamesperyear = Yankees_data.groupby(Yankees_data.index.year)['GameType'].count().values ## Determine amount of games/year
Yankees_data['GameNumber'] = ""
for i, year in enumerate(Yankees_data.index.year.unique().values):
    Yankees_data.loc[Yankees_data.index.year == year, 'GameNumber'] = list(range(1,gamesperyear[i]+1))

In [224]:
## WinnerNumber column is useful for calculating WinLossRatio (next cell)
Yankees_data['WinnerNumber'] = ""
Yankees_data.loc[Yankees_data['Winner'] == 'W', 'WinnerNumber'] = 1
Yankees_data.loc[Yankees_data['Winner'] == 'L', 'WinnerNumber'] = -1
Yankees_data.loc[Yankees_data['PointDifferential'] == 0, 'WinnerNumber'] = 0

In [225]:
## Create Winlossratio column which calculates win/loss for all prior games in a season
## Also creates WinLossRatioLast10 which is win/loss ratio for last 10 games

Yankees_data['WinLossRatio'] = ''
Yankees_data['WinLossRatioLast10'] = ''

for j, year in enumerate(Yankees_data.index.year.unique().values):
    i = gamesperyear[j]
    ## Win/loss for all season
    Yankees_data.loc[(Yankees_data.index.year == year) & (Yankees_data['GameNumber'] == 1),'WinLossRatio'] = 0
    Yankees_data.loc[(Yankees_data.index.year == year) & (Yankees_data['GameNumber'].isin(list(range(2,i+1)))),'WinLossRatio'] = Yankees_data.loc[Yankees_data.index.year == year,'WinnerNumber'].cumsum()[:-1].values
    
    ## Win/loss for last 10 games
    Yankees_data.loc[(Yankees_data.index.year == year) & (Yankees_data['GameNumber'] == 1),'WinLossRatioLast10'] = 0
    Yankees_data.loc[(Yankees_data.index.year == year) & (Yankees_data['GameNumber'].isin(list(range(2,i+1)))),'WinLossRatioLast10'] = Yankees_data.loc[Yankees_data.index.year == year,'WinnerNumber'].rolling(min_periods=1, window=10).sum()[:-1].values


In [226]:
Yankees_data

,GameType,DayOfWeek,VisitingTeam_Team,VisitingTeam_League,HomeTeam,DayNight,Completion,Attendance,VisitingTeam_Hits,HomeTeam_Hits,VisitingScore,HomeScore,PointDifferential,Winner,GameNumber,WinnerNumber,WinLossRatio,WinLossRatioLast10
0,,,,,,,,,,,,,,,,,,
1978-04-07,0,Fri,TOR,AL,DET,D,NaN,52528.0,0.0,1.0,2,6,-4.0,L,1,-1,0,0
1978-04-08,0,Sat,TOR,AL,DET,D,NaN,12413.0,1.0,2.0,5,2,3.0,W,2,1,-1,-1
1978-04-09,0,Sun,TOR,AL,DET,D,NaN,9169.0,2.0,1.0,4,8,-4.0,L,3,-1,0,0
1978-04-11,0,Tue,TOR,AL,CHA,D,NaN,3347.0,1.0,1.0,5,9,-4.0,L,4,-1,-1,-1
1978-04-12,0,Wed,TOR,AL,CHA,D,NaN,5742.0,2.0,1.0,4,5,-1.0,L,5,-1,-2,-2
1978-04-14,0,Fri,DET,AL,TOR,D,NaN,35761.0,1.0,2.0,8,10,2.0,W,6,1,-3,-3
1978-04-15,0,Sat,DET,AL,TOR,D,NaN,15024.0,2.0,1.0,6,3,-3.0,L,7,-1,-2,-2
1978-04-16,0,Sun,DET,AL,TOR,D,NaN,17088.0,1.0,2.0,4,3,-1.0,L,8,-1,-3,-3
1978-04-17,0,Mon,KCA,AL,TOR,D,NaN,10169.0,1.0,2.0,3,1,-2.0,L,9,-1,-4,-4


In [227]:
Yankees_data.to_csv('BlueJays.csv')